# py 3.12, torch 2.6 e o transformers 4.52.4

In [ ]:
!pip install --quiet transformers[torch] sentencepiece datasets evaluate

#conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

^C


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.
torchvision 0.20.1 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.


# A biblioteca transformers :

# Pipeline seleciona um modelo pre treinado, faz download, tokeniza , roda o modelo encima do texto desejado e depois faz o pos processamento. 

In [2]:
from transformers import pipeline

analysis = pipeline("sentiment-analysis")

analysis("I feel so nice")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998635053634644}]

In [3]:
analysis("I am waiting for my grades")

[{'label': 'NEGATIVE', 'score': 0.9972702860832214}]

In [4]:
analysis("I would like to study quantum physics")

[{'label': 'POSITIVE', 'score': 0.9972067475318909}]

# Eh possivel verificar a documentacao desse modelo  do hugging face: https://huggingface.co/tabularisai/multilingual-sentiment-analysis

In [5]:
class_multilanguage = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

class_multilanguage("Hoje eh segunda feira")

Device set to use cuda:0


[{'label': 'Neutral', 'score': 0.2815445363521576}]

In [6]:
class_multilanguage("Estou feliz")

[{'label': 'Very Positive', 'score': 0.6657947897911072}]

In [7]:
from transformers import AutoTokenizer

check = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(check)

tokens = tokenizer("I've been waiting for this my whole life.")

print(tokens)

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 2023, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


# Classificacao zero-shot: permite classificar sem necessidade de exemplos rotulados no conjunto de dados treinados. Utiliza modelos que ja conhecem vasto vocabulario e conceitos, exemplo disso eh Transfer Learning.

In [8]:
classificador_tf=pipeline("zero-shot-classification")

result = classificador_tf("This is a course about transformers library",
                          candidate_labels = ["education", "politics", "economy", "movies"])
import pandas as pd 

pd.DataFrame(result)



No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


,sequence,labels,scores
0,This is a course about transformers library,education,0.867794
1,This is a course about transformers library,economy,0.064698
2,This is a course about transformers library,movies,0.043550
3,This is a course about transformers library,politics,0.023957


# Pode ser usado para completar texto tambem, utiliza o GPT 2 pois era open source

In [9]:
generator = pipeline("text-generation")

generator("In this course, we will teach how to ")

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach how to \xa0create a simple, easy to use and very intuitive 3D model of a real human body.\nBuilding the Body\nThe body is a complex, complex, complex object that is not always easy to understand. In order to understand how to construct and create a 3D model of our body, we will need to understand how to build the body by moving and moving.\nWhen building a 3D model, you also need to understand how much energy is required to build a 3D model. In order to build a 3D model, you also need to understand the basic principles of drawing and drawing an object.\nTo build a 3D model, you also need to understand how to create an object.\nWe will start with creating a body object. As I mentioned in the beginning, this is just the basic object which we will use to build the 3D model.\nYou will now start to create an object of the following types:\nBody\nBody with a unique shape.\nBody with a unique shape of varying shape.\nBody with a unique shape 

In [10]:
import torch
print(f"Is CUDA available? {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Is CUDA available? True
Using device: cuda


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TucanoBR/Tucano-630m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

text = "Olá, como você está?"
inputs = tokenizer(text, return_tensors="pt")
output = model.generate(**inputs)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Olá, como você está? Eu sou o seu novo amigo. Sou um gato de rua e estou procurando alguém para me adotar, mas não sei por onde começar. Você pode me ajudar com isso?

Claro, ficarei feliz em ajudá-lo a encontrar uma pessoa para adotar! Aqui estão algumas etapas que você pode seguir:

1. Verifique as redes sociais: procure grupos ou páginas locais de adoção de animais nas redes sociais. Eles geralmente têm informações sobre abrigos locais, organizações sem fins lucrativos e outros recursos.

2. Entre em contato com abrigos: entre em contato com abrigos locais ou organizações sem fins lucrativos que aceitam animais de estimação e pergunte se eles podem fornecer algum tipo de abrigo para animais de estimação. Muitas vezes, os abrigos oferecem serviços de acolhimento temporário, cuidados veterinários gratuitos e outras formas de apoio.

3. Pergunte ao seu médico de atenção primária: Seu médico de atenção primária, também conhecido como clínico geral, é responsável por encaminhá-lo aos abr

In [13]:
from transformers import pipeline

translator = pipeline("translation_en_to_pt", model="unicamp-dl/translation-en-pt-t5")

# Tradução do inglês para o português
entrada = "Translate from English to Portuguese: The weather is nice today."
resultado = translator(entrada, max_length=50)

print(resultado)

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'translation_text': 'O clima é bom hoje.'}]
